In [1]:
import os
import sys
import itertools as itt

# File reading
from pathlib import Path
import czifile as czi
import tifffile as tiff

# Image preprocessing
import cv2
from skimage.filters import threshold_li, threshold_minimum, threshold_triangle
from skimage.morphology import remove_small_objects


# Calculations
from scipy import ndimage as ndi
import numpy as np

# Analysis
import astrobject as ao
import ccdb
import astromorpho as astro

# Viewing
import napari
import matplotlib.pyplot as plt
%matplotlib inline

# Beauty
from tqdm.auto import tqdm

Can't load imreg package, affine and homography registrations won't work


/home/incredible/Neuroscience/Code/lib/imfun/imfun/external/tifffile.py:7261: UserWarning: No module named 'imfun.external._tifffile'
  Functionality might be degraded or be slow.

  warnings.warn("%s%s" % (e, warn))
/home/incredible/Neuroscience/Code/lib/imfun/imfun/external/czifile.py:146: UserWarning: ImportError: No module named '_czifile'. Decoding of JXR and JPEG encoded images will not be available. Czifile.pyx can be obtained at http://www.lfd.uci.edu/~gohlke/
  warnings.warn(


# Параметры для запуска

In [2]:
if os.path.exists('/home/incredible/astro-morpho'):
    data_dir = '/home/incredible/astro-morpho/data/HT_Tumor_12vs34 for Tatiana/'
else:
    print("Dont know where to look for the data")

output_dir = '../output/'

filename = 'T1.czi'

use_clahe = True
sigmas = 2**np.arange(0, 3, 0.5)
alpha = 0.5 # relative weight of coside distance between Hessian vector and linkage vector
beta = 0.0  # weight of Euclidean distance between the nodes 
offset=1

VERBOSE = True

# Set false to start from console
HANDY = True

# Set true to save output
OUT = False

# Считывание изображения

In [3]:
if HANDY:
    VERBOSE = True

In [4]:
datapath = Path(data_dir).joinpath(filename)
# datapath = Path(data_dir).joinpath(filename)
datapath

PosixPath('/home/incredible/astro-morpho/data/HT_Tumor_12vs34 for Tatiana/T1.czi')

In [5]:
tifname = str(datapath)[:-3] + 'tif' 

czi.czi2tif(datapath, tiffile=tifname)
stack = tiff.imread(tifname)[2]


Opening CZI file... 0.605424 s
Image
  axes:  CZYX
  shape: (3, 70, 512, 512)
  dtype: uint16
  size:  105.00 MiB
Creating empty TIF file... 0.135808 s
Copying image from CZI to TIF file... 3.477781 s


In [6]:
if VERBOSE:
    w = napari.view_image(stack)

KeyboardInterrupt: 

# Удаление пипетки

In [27]:
import pipette_segmentation as ps

In [ ]:
simple_mask = ps.make_simple_mask(stack)
sato_mask = ps.make_sato_mask(stack)
masks3 = ps.combine_masks(stack, simple_mask, sato_mask)

vx = masks3.sum(axis=(1,2))
vx_sato = ps.st_roll(sato_mask).sum(axis=(1,2))

try:
    kink = ps.find_kink(vx)
except :
    #print(E)
    kink = len(vx)

masks3a = masks3.copy()
masks3a[kink:]=0
masks3a = ucats.masks.largest_region(masks3a)

masks3_final = masks3a if use_kink else masks3
masks3_final = np.moveaxis(masks3_final,0,2)

show_stack = (stack*(~(ndi.binary_dilation(masks3_final,iterations=3)))).astype(float64)